In [1]:
import pandas as pd
import numpy as np

In [25]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [3]:
import yfinance as yf

In [4]:
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings("ignore")

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", 100)

In [6]:
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

In [7]:
config_params = {
    "Tickers": [
        # Technology
        "AAPL",
        "MSFT",
        "GOOGL",
        "AMZN",
        "TSLA",
        "NVDA",
        "META",
        # Financial
        "BRK-B",
        "JPM",
        "V",
        # Healthcare
        "JNJ",
        "LLY",
        "UNH",
        # Consumer Discretionary
        "PG",
        "WMT",
        "COST",
        # Energy
        "XOM",
        "CVX",
        # Benchmark
        "SPY",
    ],
    "start_date": "2015-01-01",
    "end_date": "2026-01-01",
    "risk_free_rate": 4.18,
    "rebalance_frequency": "quarterly",
    "transaction_cost": 0.0015,
    "train_test_split": 0.8,
    "validation_split": 0.2,
    "radom_seed": 42,
}

In [8]:
def download_data(tickers, start_date, end_date, progress=True):
    print("Downloading stock data from Yahoo Finance...")
    print(f"Tickers: {', '.join(tickers)}")

    try:
        data = yf.download(
            tickers=tickers,
            start=start_date,
            end=end_date,
            progress=progress,
            group_by="tickers",
            auto_adjust=True,
        )

        if data.empty:
            raise ValueError(
                "No data downloaded. Please check the tickers and date range."
            )
        return data

    except Exception as e:
        print(f"Error downloading data: {str(e)}")
        return None

In [9]:
def process_raw_data(raw_data, tickers):
    if len(tickers) == 1:
        prices = raw_data["Close"].to_frame()
        prices.columns = tickers
        volume = raw_data["Volume"].to_frame()
        volume.columns = tickers
    else:
        prices = pd.DataFrame()
        volume = pd.DataFrame()

        for ticker in tickers:
            try:
                if isinstance(raw_data.columns, pd.MultiIndex):
                    prices[ticker] = raw_data[ticker]["Close"]
                    volume[ticker] = raw_data[ticker]["Volume"]
                else:
                    prices[ticker] = raw_data[ticker]
                    volume[ticker] = raw_data[ticker]
            except KeyError:
                print(f"Warning: No data found for ticker {ticker}. Skipping.")

    returns = prices.pct_change()
    returns = returns.dropna()

    print("Processed data:")
    print(f"Prices shape: {prices.shape}")
    print(f"Returns shape: {returns.shape}")
    print(f"Missing values in prices: {prices.isna().sum().sum()}")
    print(f"Missing values in returns: {returns.isna().sum().sum()}")

    return {"prices": prices, "returns": returns, "volume": volume}


In [10]:
raw_data = download_data(
    tickers=config_params["Tickers"],
    start_date=config_params["start_date"],
    end_date=config_params["end_date"],
)

raw_data

Tickers: AAPL, MSFT, GOOGL, AMZN, TSLA, NVDA, META, BRK-B, JPM, V, JNJ, LLY, UNH, PG, WMT, COST, XOM, CVX, SPY


[*********************100%***********************]  19 of 19 completed


Ticker            COST                                               \
Price             Open        High         Low       Close   Volume   
Date                                                                  
2015-01-02  114.345600  115.377264  113.491254  114.136047  1471000   
2015-01-05  114.200509  114.579325  112.677191  112.838387  1748500   
2015-01-06  113.330038  115.207992  112.717491  114.329468  3185200   
2015-01-07  114.933973  116.408921  114.740531  116.320274  2171600   
2015-01-08  117.319687  117.754928  116.819978  117.319687  2937700   
...                ...         ...         ...         ...      ...   
2025-12-24  858.599976  876.000000  858.349976  871.859985  1822200   
2025-12-26  870.640015  877.590027  869.000000  873.349976  1324100   
2025-12-29  873.289978  874.500000  865.690002  867.840027  1792500   
2025-12-30  864.000000  867.820007  861.429993  865.650024  1680800   
2025-12-31  864.580017  868.000000  861.340027  862.340027  1490000   

Ticker            META                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2015-01-02   78.034911   78.382481   77.161010   77.905807  18177500   
2015-01-05   77.439062   78.700249   76.326828   76.654541  26452200   
2015-01-06   76.694267   77.051763   74.837237   75.621758  27399300   
2015-01-07   76.227527   76.823363   75.294045   75.621758  22045300   
2015-01-08   76.207667   77.687336   75.552249   77.637680  23961000   
...                ...         ...         ...         ...       ...   
2025-12-24  662.530029  668.179993  662.200012  667.549988   5627500   
2025-12-26  668.059998  668.950012  661.320007  663.289978   7133800   
2025-12-29  658.010010  660.250000  654.390015  658.690002   8506500   
2025-12-30  658.690002  672.219971  657.840027  665.950012   9187500   
2025-12-31  664.750000  665.000000  659.440002  660.090027   7940400   

Ticker             JPM                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2015-01-02   46.280406   46.860957   46.198533   46.511139  12600000   
2015-01-05   46.191086   46.354830   44.829021   45.067196  20100600   
2015-01-06   45.134181   45.216054   43.429739   43.898647  29074100   
2015-01-07   44.575949   44.575949   43.667906   43.965626  23843200   
2015-01-08   44.635501   45.327698   44.635501   44.948105  16971100   
...                ...         ...         ...         ...       ...   
2025-12-24  324.774998  328.508159  324.038331  327.691864   4289300   
2025-12-26  327.632124  329.374265  325.073687  326.437531   4158300   
2025-12-29  325.531610  326.298141  322.077190  322.296204   8635300   
2025-12-30  323.351455  323.480876  321.041865  321.967712   7904300   
2025-12-31  322.883547  323.421130  320.593887  320.773071   5048500   

Ticker            NVDA                                                 \
Price             Open        High         Low       Close     Volume   
Date                                                                    
2015-01-02    0.483011    0.486611    0.475333    0.483011  113680000   
2015-01-05    0.483011    0.484451    0.472694    0.474853  197952000   
2015-01-06    0.475573    0.476053    0.459977    0.460456  197764000   
2015-01-07    0.463816    0.467895    0.457817    0.459257  321808000   
2015-01-08    0.464536    0.479412    0.464296    0.476533  283780000   
...                ...         ...         ...         ...        ...   
2025-12-24  187.940002  188.910004  186.589996  188.610001   65528500   
2025-12-26  189.919998  192.690002  188.000000  190.529999  139740300   
2025-12-29  187.710007  188.759995  185.910004  188.220001  120006100   
2025-12-30  188.240005  188.990005  186.929993  187.539993   97687300   
2025-12-31  189.570007  190.559998  18

In [11]:
processed_data = process_raw_data(raw_data, config_params["Tickers"])

Processed data:
Prices shape: (2766, 19)
Returns shape: (2765, 19)
Missing values in prices: 0
Missing values in returns: 0


In [12]:
processed_data

{'prices':                   AAPL        MSFT       GOOGL        AMZN        TSLA  \
 Date                                                                     
 2015-01-02   24.237551   39.858456   26.278944   15.426000   14.620667   
 2015-01-05   23.554743   39.491928   25.778227   15.109500   14.006000   
 2015-01-06   23.556961   38.912292   25.142035   14.764500   14.085333   
 2015-01-07   23.887278   39.406681   25.068094   14.921000   14.063333   
 2015-01-08   24.805079   40.565945   25.155434   15.023000   14.041333   
 ...                ...         ...         ...         ...         ...   
 2025-12-24  273.809998  488.019989  314.089996  232.380005  485.399994   
 2025-12-26  273.399994  487.709991  313.510010  232.520004  475.190002   
 2025-12-29  273.760010  487.100006  313.559998  232.070007  459.640015   
 2025-12-30  273.079987  487.480011  313.850006  232.529999  454.429993   
 2025-12-31  271.859985  483.619995  313.000000  230.820007  449.720001   
 
             

In [13]:
prices = processed_data["prices"]
returns = processed_data["returns"]
volume = processed_data["volume"]

In [14]:
def summary_stats(returns, prices, risk_free_rate):
    """Calculate summary statistics for given returns and prices DataFrames."""

    stats = pd.DataFrame(index=returns.columns)

    # Annualized Return
    avg_daily_return = returns.mean()
    stats["annualized_return"] = ((1 + avg_daily_return) ** 252) - 1

    # Annualized Volatility
    daily_volatility = returns.std()
    stats["annualized_volatility"] = daily_volatility * np.sqrt(252)

    # Sharpe Ratio
    stats["sharpe_ratio"] = (
        stats["annualized_return"] - (risk_free_rate / 100)
    ) / stats["annualized_volatility"]

    # Max Drawdown
    mdd = []

    for ticker in prices.columns:
        running_peak = prices[ticker].cummax()
        drawdown = (prices[ticker] - running_peak) / running_peak
        mdd.append(drawdown.min())

    stats["max_drawdown"] = mdd

    stats["current_price"] = prices.iloc[-1]
    stats["start_price"] = prices.iloc[0]
    stats["total_returns"] = (prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]

    for col in [
        "annualized_return",
        "annualized_volatility",
        "max_drawdown",
        "total_returns",
    ]:
        stats[col] = stats[col].apply(lambda x: f"{x * 100:.2f}%")

    stats["sharpe_ratio"] = stats["sharpe_ratio"].apply(lambda x: f"{x:.3f}")
    stats["current_price"] = stats["current_price"].apply(lambda x: f"{x:.2f}")
    stats["start_price"] = stats["start_price"].apply(lambda x: f"{x:.2f}")

    return stats

In [15]:
summary_stats(
    returns=returns, prices=prices, risk_free_rate=config_params["risk_free_rate"]
)

,annualized_return,annualized_volatility,sharpe_ratio,max_drawdown,current_price,start_price,total_returns
AAPL,29.93%,28.85%,0.893,-38.52%,271.86,24.24,1021.65%
MSFT,30.15%,26.89%,0.966,-37.15%,483.62,39.86,1113.34%
GOOGL,30.63%,28.84%,0.917,-44.32%,313.00,26.28,1091.07%
AMZN,35.05%,32.93%,0.937,-56.15%,230.82,15.43,1396.31%
TSLA,61.28%,57.75%,0.989,-73.63%,449.72,14.62,2975.92%
NVDA,93.47%,48.69%,1.834,-66.34%,186.50,0.48,38511.92%
META,30.44%,37.53%,0.700,-76.74%,660.09,77.91,747.29%
BRK-B,13.80%,19.27%,0.499,-29.57%,502.65,149.17,236.96%
JPM,23.70%,27.15%,0.719,-43.63%,320.77,46.51,589.67%
V,20.72%,24.26%,0.682,-36.36%,350.71,61.34,471.74%


In [16]:
returns

,AAPL,MSFT,GOOGL,AMZN,TSLA,NVDA,META,BRK-B,JPM,V,JNJ,LLY,UNH,PG,WMT,COST,XOM,CVX,SPY
Date,,,,,,,,,,,,,,,,,,,
2015-01-05,-0.028171,-0.009196,-0.019054,-0.020517,-0.042041,-0.016890,-0.016061,-0.014547,-0.031045,-0.022074,-0.006984,-0.009848,-0.016471,-0.004754,-0.002910,-0.011369,-0.027361,-0.039971,-0.018060
2015-01-06,0.000094,-0.014677,-0.024679,-0.022833,0.005664,-0.030318,-0.013473,-0.001088,-0.025929,-0.006443,-0.004914,0.005045,-0.002018,-0.004555,0.007706,0.013214,-0.005317,-0.000463,-0.009419
2015-01-07,0.014022,0.012705,-0.002941,0.010600,-0.001562,-0.002605,0.000000,0.013893,0.001526,0.013398,0.022076,-0.007028,0.010210,0.005246,0.026532,0.017413,0.010133,-0.000833,0.012461
2015-01-08,0.038422,0.029418,0.003484,0.006836,-0.001564,0.037618,0.026658,0.016725,0.022347,0.013412,0.007863,0.023689,0.047733,0.011435,0.021106,0.008592,0.016644,0.022883,0.017745
2015-01-09,0.001073,-0.008405,-0.012211,-0.011749,-0.018802,0.004028,-0.005628,-0.012552,-0.017387,-0.014823,-0.013629,-0.013264,-0.009360,-0.009330,-0.012380,-0.015389,-0.001410,-0.019926,-0.008014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-24,0.005324,0.002403,-0.000827,0.001034,-0.000330,-0.003171,0.003925,0.001658,0.009941,0.004981,0.009719,0.004983,0.008559,0.009149,0.006402,0.019970,-0.001675,-0.000066,0.003518
2025-12-26,-0.001497,-0.000635,-0.001847,0.000602,-0.021034,0.010180,-0.006382,-0.006064,-0.003828,-0.000394,-0.000722,0.000715,0.012974,0.001730,0.001165,0.001709,-0.000923,-0.003189,-0.000101
2025-12-29,0.001317,-0.001251,0.000159,-0.001935,-0.032724,-0.012124,-0.006935,0.005519,-0.012686,-0.001099,-0.000337,0.000909,-0.008709,-0.001175,0.007070,-0.006309,0.011922,0.006466,-0.003564


In [ ]:
corr_matrix = np.round(returns.corr(), 2)

In [29]:
corr_matrix

,AAPL,MSFT,GOOGL,AMZN,TSLA,NVDA,META,BRK-B,JPM,V,JNJ,LLY,UNH,PG,WMT,COST,XOM,CVX,SPY
AAPL,1.00,0.66,0.59,0.55,0.43,0.53,0.52,0.51,0.42,0.56,0.32,0.28,0.33,0.35,0.32,0.47,0.31,0.34,0.75
MSFT,0.66,1.00,0.68,0.65,0.41,0.60,0.59,0.51,0.44,0.61,0.33,0.31,0.36,0.38,0.33,0.50,0.27,0.34,0.79
GOOGL,0.59,0.68,1.00,0.63,0.38,0.52,0.61,0.45,0.41,0.53,0.27,0.26,0.31,0.27,0.25,0.40,0.27,0.31,0.71
AMZN,0.55,0.65,0.63,1.00,0.40,0.51,0.60,0.35,0.31,0.45,0.18,0.22,0.23,0.20,0.25,0.42,0.19,0.21,0.64
TSLA,0.43,0.41,0.38,0.40,1.00,0.41,0.34,0.27,0.28,0.34,0.09,0.12,0.16,0.11,0.17,0.28,0.17,0.22,0.49
NVDA,0.53,0.60,0.52,0.51,0.41,1.00,0.48,0.35,0.34,0.45,0.13,0.22,0.23,0.17,0.20,0.37,0.20,0.25,0.65
META,0.52,0.59,0.61,0.60,0.34,0.48,1.00,0.35,0.33,0.45,0.18,0.23,0.20,0.20,0.21,0.36,0.19,0.23,0.61
BRK-B,0.51,0.51,0.45,0.35,0.27,0.35,0.35,1.00,0.75,0.63,0.51,0.33,0.45,0.47,0.38,0.43,0.56,0.59,0.76
JPM,0.42,0.44,0.41,0.31,0.28,0.34,0.33,0.75,1.00,0.58,0.37,0.25,0.37,0.32,0.26,0.32,0.54,0.58,0.72
V,0.56,0.61,0.53,0.45,0.34,0.45,0.45,0.63,0.58,1.00,0.41,0.31,0.41,0.42,0.30,0.43,0.42,0.46,0.76


In [30]:
x_lables = [f"{i}" for i in returns.columns]
y_lables = [f"{i}" for i in returns.columns]
data = corr_matrix

trace = go.Heatmap(z=data, x=x_lables, y=y_lables)

fig = go.Figure(data=[trace])

fig.update_layout(title="Portfolio's Correlation")

fig.show()

In [ ]:
corr_matrix.iloc[1, 1]

np.float64(1.0)

In [47]:
def correlation_analysis(returns):
    corr_matrix = returns.corr()

    print("Correlation Heatmap")

    fig = px.imshow(
        corr_matrix,
        labels=dict(color="Correlation Value"),
        title="Portfolio Correlation",
        text_auto=True,
        aspect="auto",
    )

    high_corr = []

    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] > 0.7:
                high_corr.append(
                    {
                        "stock 1": corr_matrix.columns[i],
                        "stock 2": corr_matrix.columns[j],
                        "correlation": corr_matrix.iloc[i, j],
                    }
                )

    if high_corr:
        high_corr_df = pd.DataFrame(high_corr).sort_values(
            "correlation", ascending=False
        )
        print(high_corr_df.to_string(index=False))
    else:
        print("There are no pairs with correlation > 0.7")

    low_corr = []

    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] < 0.3:
                low_corr.append(
                    {
                        "stock 1": corr_matrix.columns[i],
                        "stock 2": corr_matrix.columns[j],
                        "correlation": corr_matrix.iloc[i, j],
                    }
                )

    if low_corr:
        low_corr_df = pd.DataFrame(low_corr).sort_values("correlation")
        print(low_corr_df.to_string(index=False))
    else:
        print("There are no pairs with correlation < 0.3")

    return corr_matrix, fig


In [ ]:
correlation_analysis(returns)
fig.show()